# [kaggle][学习向]sf-crime数据的传统机器学习（其一）

这是一篇专门用来灌水的部分，在这一篇里面，我们将使用传统机器学习的方法建立sf-crime的数据模型

**灌水部分**，不调参，不提交，只灌水

导入数据

In [1]:
import numpy as np
from sklearn.metrics import log_loss

train_features = np.load('./data/train_features.npy')
train_labels = np.load('./data/train_labels.npy')
test_features = np.load('./data/test_features.npy')

## 朴素贝叶斯

In [2]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
model.fit(train_features, train_labels)
predicted = np.array(model.predict_proba(train_features))
print ("朴素贝叶斯的log损失为 %f" % (log_loss(train_labels, predicted)))

朴素贝叶斯的log损失为 2.582578


## 逻辑回归

In [3]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.1)
model.fit(train_features, train_labels)
predicted = np.array(model.predict_proba(train_features))
print ("逻辑回归的log损失为 %f" % (log_loss(train_labels, predicted)))

C:\Users\XPS\.conda\envs\Pytorch-learn\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


逻辑回归的log损失为 2.540033


## 决策树

易过拟合

In [4]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth = 5)
model.fit(train_features, train_labels)
predicted = np.array(model.predict_proba(train_features))
print ("决策树的log损失为 %f" % (log_loss(train_labels, predicted)))

决策树的log损失为 2.543998


## 随机森林

易内存不足，且有过拟合现象

In [5]:
from sklearn.ensemble import RandomForestClassifier  
model = RandomForestClassifier(max_leaf_nodes = 5,n_estimators = 10)
#随手写的参数
model.fit(train_features, train_labels)
predicted = np.array(model.predict_proba(train_features))
print ("随机森林的log损失为 %f" % (log_loss(train_labels, predicted)))

随机森林的log损失为 2.597370


## 梯度提升决策树

以lightGBM为例

In [6]:
import lightgbm as lgb
model = lgb.LGBMClassifier(num_leaves=35,learning_rate=0.05,n_estimators=20)
#随手写的参数
model.fit(train_features, train_labels)
predicted = np.array(model.predict_proba(train_features))
print ("lightGBM的log损失为 %f" % (log_loss(train_labels, predicted)))

lightGBM的log损失为 2.509984


In [ ]:
import pandas as pd
testResult = np.array(model.predict_proba(test_features))
sampleSubmission = pd.read_csv('../input/sf-crime/sampleSubmission.csv.zip')
Result_pd = pd.DataFrame(testResult,
                         index=sampleSubmission.index,
                         columns=sampleSubmission.columns[1:])
Result_pd.to_csv('../working/sampleSubmission(test).csv', index_label='Id')

提交

由于数据集的问题，SVM很难找到最优超平面，遂舍去。

后面会使用神经网络，所以逻辑回归不继续调优。

梯度提升决策树将在神经网络后面进行调优。